In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install langchain_community
!pip install pypdfium2
!pip install langchain_experimental
!pip install kiwipiepy
!pip install faiss-gpu-cu12
!pip install rank_bm25

In [ ]:
import os
from langchain.docstore.document import Document

root_folder = "C:/Users/minkyu/Desktop/dacon accident prevention/진짜 시작/건설안전지침_md"

all_documents = []  # Document 객체 리스트

# 폴더를 순회하며 .md 파일 처리
for current_path, dirs, files in os.walk(root_folder):
    # 현재 경로가 루트 폴더와 다른 경우(=서브폴더), 폴더명을 메타데이터 title로 사용
    if current_path != root_folder:
        folder_title = os.path.basename(current_path)
    else:
        folder_title = ""

    # 현재 폴더의 .md 파일 목록
    md_files = [f for f in files if f.lower().endswith('.md')]

    # page 번호를 추출해 정렬하기 위한 헬퍼 함수
    def extract_page_number(filename):
        # 예: "page_1.md" -> "page_1" -> 뒤의 "1" -> int(1)
        base = os.path.splitext(filename)[0]  # "page_1"
        return int(base.split('_')[-1])       # "1"

    # .md 파일 정렬
    md_files = sorted(md_files, key=extract_page_number)

    # 첫 3개(page_1, page_2, page_3)는 무시하고, 그 이후 파일들만
    md_files_to_read = md_files[3:]

    # 페이지 단위로 Document 생성
    for i, md_file in enumerate(md_files_to_read, start=4):  # 시작 인덱스를 4로 설정
        md_file_path = os.path.join(current_path, md_file)
        with open(md_file_path, "r", encoding="utf-8") as f:
            page_content = f.read()

        # 메타데이터에 폴더명과 실제 페이지 번호(i)를 저장
        doc_metadata = {
            "title": folder_title if folder_title else "NoTitle",
            "page_num": i
        }

        doc = Document(page_content=page_content, metadata=doc_metadata)
        all_documents.append(doc)

        print(f"폴더: {folder_title}, 파일: {md_file}, 페이지 번호: {i}")

print(f"\n총 Document 수: {len(all_documents)}")


In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")

content_index = FAISS.from_documents(all_documents, embeddings)
content_index.save_local("faiss_content_index")

print("FAISS content index saved successfully.")

In [ ]:
import pandas as pd
train_dataset = pd.read_csv('train_df.csv')

In [ ]:
combined_training_data = train_dataset.apply(
    lambda row: {
        "text": (
            f"{row['인적사고']} 에 대한 안전조치사항 "
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

In [ ]:
print(combined_training_data[0]['text'])


In [ ]:
# bm25 토크나이저로 한국어 토크나이저 사용 하기 위해서 불러옴
from kiwipiepy import Kiwi
kiwi = Kiwi()

def ko_kiwi_tokenizer(text: str):
    # Kiwi 토크나이저는 각 토큰에 대한 다양한 정보를 반환합니다.
    # 여기서는 토큰의 표면 형태(텍스트)만 추출합니다.
    tokens = kiwi.tokenize(text)
    return [token[0] for token in tokens]

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.docstore.document import Document
# Retriever 정의
# pdf_chunks에서 Document 객체 리스트 생성
# pdf_chunks에서 Document 객체 리스트 생성
docs = all_documents
retriever = content_index.as_retriever(search_type="similarity", search_kwargs={"k": 3})
bm25_retriever = BM25Retriever.from_documents(docs, tokenizer=ko_kiwi_tokenizer , k =3 )

# 앙상블 리트리버 생성
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever],
    weights=[0.5, 0.5]  # 각 리트리버에 동일 가중치 부여 (가중치 합은 1.0)
)

In [ ]:
combined_training_data[1]['text']

In [ ]:
import numpy as np

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
n=9

# query와 정답 출력
query = combined_training_data[n]['text']
print('질문 :', query)
print('정답 :', combined_training_data[n]['answer'])

# query 임베딩 계산
query_embedding = embeddings.embed_query(query)

# 앙상블 리트리버로 검색 후 각 문서에 대해 코사인 유사도 계산 및 출력
results = ensemble_retriever.get_relevant_documents(query)

for doc in results:
    # 문서의 내용을 일부 출력
    
    # 문서 임베딩 계산 (문서 전체를 대상으로 임베딩을 생성합니다)
    doc_embedding = embeddings.embed_query(doc.page_content)
    similarity = cosine_similarity(query_embedding, doc_embedding)
    
    print(f"\n제목: {doc.metadata.get('title')}")
    print("내용 일부:", doc.page_content)
    print("코사인 유사도:", similarity)
